In [1]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
# read csv into pandas dataframe
df = pd.read_csv('withdays.csv')
df['category'] = df['category'].map({
'Graphic Novels & Comics' : 0,
'Young Adult Fiction' : 1,
'Memoir & Autobiography' : 2,
'Picture Books' : 3,
'Romance' : 4,
'Humor' : 5,
'Poetry' : 6,
'Horror' : 7,
'Young Adult Fantasy' : 8,
'Science Fiction' : 9,
"Middle Grade & Children's" : 10,
'History & Biography' : 11,
'Nonfiction' : 12,
'Fantasy': 13,
'Mystery & Thriller' : 14,
'Historical Fiction' : 15,
'Debut Goodreads Author' : 16,
'Fiction' : 17,
'Paranormal Fantasy' : 18,
'Food & Cookbooks' : 19,
'Business Books' : 20,
'Science & Technology' : 21,
'Goodreads Author' : 22,
'Debut Novel' : 23,
'Best of the Best' : 24,
'Travel & Outdoors' : 25,
'Food & Cooking' : 19,
'Favorite Book of 2011' : 24,
'Debut Author' : 26
})
df['category'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 26, 24, 25])

In [3]:
df['name'].nunique()

3113

In [4]:
dups = df[df.duplicated(['name', 'year', 'category'], keep=False)]
dups

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,year,name,writer,category,winner,num 1 stars,num 2 stars,...,average_rating_w,num_ratings,num_reviews,url,daystocom,isbn,times_appred_NYTime,avgrank_NYTime,bestRank,listnames
2016,2016,2016,2016,2018,The Astonishing Color of After,['Emily X.R. Pan'],1,0,0.013841,0.013841,...,4.13,15330.0,3490.0,https://www.goodreads.com/book/show/35604686-t...,259.0,031646399X,1.0,40.000000,10.0,1.0
2022,2022,2022,2022,2018,The Astonishing Color of After,['Emily X.R. Pan'],1,0,0.013793,0.013793,...,4.13,15330.0,3490.0,https://www.goodreads.com/book/show/35604686-t...,259.0,031646399X,1.0,40.000000,10.0,1.0
2077,2077,2077,2077,2018,The Kiss Quotient,['Helen Hoang'],4,1,0.010169,0.006780,...,3.91,126620.0,20309.0,https://www.goodreads.com/book/show/36577586-t...,182.0,0451490800,0.0,inf,inf,0.0
2085,2085,2085,2085,2018,The Kiss Quotient,['Helen Hoang'],4,1,0.010169,0.006780,...,3.91,126620.0,20309.0,https://www.goodreads.com/book/show/36577586-t...,182.0,0451490800,0.0,inf,inf,0.0
2185,2185,2185,2185,2018,The Kiss Quotient,['Helen Hoang'],26,0,0.010169,0.006780,...,3.91,126620.0,20309.0,https://www.goodreads.com/book/show/36577586-t...,182.0,0451490800,0.0,inf,inf,0.0
2189,2189,2189,2189,2018,The Astonishing Color of After,['Emily X.R. Pan'],26,0,0.013841,0.013841,...,4.13,15330.0,3490.0,https://www.goodreads.com/book/show/35604686-t...,259.0,031646399X,1.0,40.000000,10.0,1.0
2195,2195,2195,2195,2018,Something in the Water,['Catherine Steadman'],26,0,0.010067,0.080537,...,3.60,76953.0,8517.0,https://www.goodreads.com/book/show/36388243-s...,182.0,1524797189,13.0,0.772173,1.0,3.0
2202,2202,2202,2202,2018,Something in the Water,['Catherine Steadman'],26,0,0.010067,0.077181,...,3.60,76953.0,8517.0,https://www.goodreads.com/book/show/36388243-s...,182.0,1524797189,13.0,0.772173,1.0,3.0
2203,2203,2203,2203,2018,The Kiss Quotient,['Helen Hoang'],26,0,0.010169,0.006780,...,3.91,126620.0,20309.0,https://www.goodreads.com/book/show/36577586-t...,182.0,0451490800,0.0,inf,inf,0.0
2204,2204,2204,2204,2018,The Astonishing Color of After,['Emily X.R. Pan'],26,0,0.013793,0.013793,...,4.13,15330.0,3490.0,https://www.goodreads.com/book/show/35604686-t...,259.0,031646399X,1.0,40.000000,10.0,1.0


In [16]:
data_test = df.loc[(df['year'].isin([2018]))]
# first method : use cathegory as a feature
# Use 2011 - 2017 as train data and 2018 as test data
logreg = LogisticRegression(solver='liblinear', class_weight = 'balanced',dual='false')

for year in [2011, 2012, 2013, 2014, 2015, 2016, 2017]:
    print(year)
    data_temp = df.loc[~(df['year'].isin([2018,2019]))]
    data_train = data_temp.loc[(data_temp['year'].isin([year]))]
    X_train = data_train[[ 'category', 'average rating', 'num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars']]
    y_train = data_train['winner']
    logreg.fit(X_train, y_train)
    
X_test = data_test[['num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars', 'category', 'average rating']]
y_test = data_test['winner']

2011


ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='logistic_regression' are not supported when dual=false, Parameters: penalty='l2', loss='logistic_regression', dual='false'

In [17]:
y_pred = logreg.predict(X_test)
predictions = pd.DataFrame(y_pred)
predictions[0].value_counts()

NotFittedError: This LogisticRegression instance is not fitted yet

In [8]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.86      0.90       403
           1       0.05      0.14      0.08        21

    accuracy                           0.83       424
   macro avg       0.50      0.50      0.49       424
weighted avg       0.91      0.83      0.86       424



In [9]:
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
print('F1 score:', metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))

Accuracy:  0.8254716981132075
F1 score: 0.8626044123427672


In [200]:
#y_new =  pd.DataFrame(y_pred)
#y_test['preds'] = y_pred
#df_out = pd.merge(df,y_test[['preds']], how = 'left',left_index = True, right_index = True)

In [10]:
# first method : use cathegory as a feature
# Use 2011 - 2017 as train data and 2018 as test data
rfc = RandomForestClassifier(n_estimators=10, class_weight = 'balanced')

for year in [2011, 2012, 2013, 2014, 2015, 2016, 2017]:
    print(year)
    data_temp = df.loc[~(df['year'].isin([2018,2019]))]
    data_train = data_temp.loc[(data_temp['year'].isin([year]))]
    X_train = data_train[[ 'category', 'average rating', 'num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars']]
    y_train = data_train['winner']
    rfc.fit(X_train, y_train)
    
X_test = data_test[['num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars', 'category', 'average rating']]
y_test = data_test['winner']
rfc_pred = rfc.predict(X_test)
metrics.accuracy_score(y_test, rfc_pred)
print(classification_report(y_test, rfc_pred))

2011
2012
2013
2014
2015
2016
2017
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       403
           1       0.00      0.00      0.00        21

    accuracy                           0.95       424
   macro avg       0.48      0.50      0.49       424
weighted avg       0.90      0.95      0.93       424



/Users/md/Desktop/collage/goodreads/task/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
